In [1]:
from langchain.agents import tool
from pydantic.v1 import BaseModel, Field
from langchain.tools.render import format_tool_to_openai_function
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json
from text2speech import *
# from langchain_anthropic import ChatAnthropic
import os 
from dotenv import load_dotenv
load_dotenv()
#os.getenv("OPENAI_API_KEY")
#os.environ["OPENAI_API_KEY"] = ""

## Define knowledge LLM's function as a tool

In [9]:
movie="Harry Potter"
chapter= None
initial_context= None
subject= "Physics"
chat_history= None

movie_mapping= {"Harry Potter": "Dumbledore"}

# Defining the knowledge_llm_func's input schema
class knowledge_llm_func_schema(BaseModel):
    user_query: str= Field(...,description= "Query asked by the user")
    subject: str= Field(...,description="Subject name")
    chapter: str= Field(...,description="Chapter name")


@tool(args_schema=knowledge_llm_func_schema)
def knowledge_llm_calling(user_query: str, subject= subject, chapter= chapter) -> str:
# def knowledge_llm_calling(user_query: str, subject_chapter_tuple: list) -> str:
    """A function that can only answer questions on Physics subject on the Newton's Law of Motion chapter"""
    return "TEST"

In [10]:
# For reply with 
story_mode= ChatOpenAI(#model= ,
    openai_api_key= os.getenv("OPENAI_API_KEY"), 
    temperature=0, 
    #prompt=prompt
    )

def initial_user_input(st_name, subject_user, chapter_user, theme):
    global subject, chapter, movie, initial_context, chat_history
    subject= subject_user
    chapter= chapter_user
    movie= theme
    initial_context= knowledge_llm_calling({'user_query': "Explain in detail, what is {chapter}? ", 'subject': subject, 'chapter': chapter})
    prompt= ChatPromptTemplate.from_messages([
    ("user", 
        f"""
    Imagine you are {movie_mapping[movie]} from the {movie} movie and you are going to teach me (your student) about {chapter}. Use the below context to teach:

    Context: {initial_context}

    Your teaching style:
    * Teach concepts by narrating a story happening in the Harry Potter universe using characters from the movie.
    * Use Dumbledore's slang and narrate the story from Dumbledore's POV.
    * Treat me as a 10 year old kid
    * Narrate a story and ask questions in between to guide me in the right path to learn the concepts. Occasionally, pause and conduct easy quiz to make sure I've understood the concepts correctly and  re-explain using if I'm wrong.
    * After asking a question, wait for my response and ONLY PROCEED AFTER GETTING THE RESPONSE.
    * Do not send more than 100 words in every iteration."""
        )
    ])
    chain_story_model= prompt | story_mode #| OpenAIFunctionsAgentOutputParser() | route_to_knowledge_llm    
    story_llm_response =chain_story_model.invoke({"chapter":chapter,"movie": movie,"initial_context":initial_context})
    chat_history = f"AI: {story_llm_response.content}"
    trasncript= story_llm_response.content
    print("trasncript:", trasncript)
    #audio_path= text_to_speech(trasncript)
    print("chat_history", chat_history)
    #print("audio_path:", audio_path)
    
   

initial_user_input("Niranjan", "Physics", "Newton's Law of Motion", "Harry Potter")

d:\Jupyter\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


trasncript: Ah, my dear student, let me tell you a tale from the magical world of Harry Potter to explain Newton's Law of Motion. Imagine Harry zooming on his broomstick during a Quidditch match. When he suddenly stops, what do you think happens according to Newton's first law? Take a moment to think and share your answer with me, young one.
chat_history AI: Ah, my dear student, let me tell you a tale from the magical world of Harry Potter to explain Newton's Law of Motion. Imagine Harry zooming on his broomstick during a Quidditch match. When he suddenly stops, what do you think happens according to Newton's first law? Take a moment to think and share your answer with me, young one.


## Define the story model's chain

In [12]:
# For reply with 
story_mode1= ChatOpenAI(#model= ,
    openai_api_key= os.getenv("OPENAI_API_KEY"), 
    temperature=0, 
    #prompt=prompt
    ).bind_tools(tools= [knowledge_llm_calling])

prompt1= ChatPromptTemplate.from_messages([
    ("system", "If the user's input is a response to the your previous question in the mentioned chat history, continue the story. If the user has asked a new question from the Newton's Law of Function, ONLY THEN call the knowledge_llm_calling function. If the user's response is irrelevant to the subject and previous conversation, remind to get back to the topic and gently continue"),
    ("user", "Chat History is: {chat_history} User's input is:{user_input}")    
])

chain_story_model1= prompt1 | story_mode1 #| OpenAIFunctionsAgentOutputParser() | route_to_knowledge_llm

# For reply with knowledge LLM's result
story_mode2= ChatOpenAI(#model= ,
    openai_api_key= os.getenv("OPENAI_API_KEY"), 
    temperature=0, 
    )

prompt2= ChatPromptTemplate.from_messages([
    ("system", "You have been teaching the user in Dumbledoor's slang and the user has asked you the below provided question. See if the below provided context can be used to answer the question. If yes, answer it. If NOT, say the teacher will address that question and continue the previous conversation."),
    ("user", "Chat History is: {chat_history} \n\n User's question is: {user_input} \n\n Context is: {new_content}")    
])

chain_story_model2= prompt2 | story_mode2


In [13]:
# Function call

def chat_user_input(user_input):
    global subject, chapter, movie, initial_context, chat_history
    # Inference 1
    story_llm_response1 =chain_story_model1.invoke({'user_input':user_input , "chapter":chapter,"subject":subject,"chat_history":chat_history})
    #story_llm_response =chain.invoke({'user_input': "I know this concept, can you teach me what's Newton's Third Law please?", "chapter":chapter,"subject":subject,"chat_history":chat_history })
    #story_llm_response = chain.invoke({'user_input': "Maybe Harry stay in the same place?", "chapter":chapter,"subject":subject,"chat_history":chat_history })

    if (story_llm_response1.content==''):
        # Call the knowledge model function
        function_name= story_llm_response1.additional_kwargs['tool_calls'][0]['function']['name']
        function_args= json.loads(story_llm_response1.additional_kwargs['tool_calls'][0]['function']['arguments'])
        #print(function_args)
        new_content= knowledge_llm_calling(function_args)
        story_llm_response2 =chain_story_model2.invoke({'user_input':user_input , 'new_content': new_content, "chapter":chapter,"subject":subject,"chat_history":chat_history })
        chat_history += f"\n\n Human:{user_input} \n\nAI: {story_llm_response2.content}"
        trasncript= story_llm_response2.content
        #audio_path= text_to_speech(trasncript)
        #print("audio_path:", audio_path)
        print("trasncript:", trasncript)
    else:
        # Update the chat history
        chat_history = str(chat_history) + f"\n\n Human:{user_input} \n\nAI: {story_llm_response1.content}"
        trasncript= story_llm_response1.content
        #audio_path= text_to_speech(trasncript)
        #print("audio_path:", audio_path)
        print("trasncript:", trasncript)

In [14]:
chat_user_input("I honestly don't know")

trasncript: It's okay if you're not sure! Let's explore this together. In the world of Harry Potter, when Harry zooms on his broomstick during a Quidditch match and suddenly stops, according to Newton's first law, what do you think would happen to him? Take a moment to think about it.


In [15]:
chat_user_input("He will remain same?")

trasncript: In the world of Harry Potter, if Harry zooms on his broomstick during a Quidditch match and suddenly stops, according to Newton's first law, he will remain in motion unless acted upon by an external force. This means that if he stops suddenly, he would continue moving forward in the air until another force, like the ground or air resistance, stops him. 

If you have any more questions or if you'd like to explore another scenario related to Newton's Laws of Motion, feel free to ask!
